In [1]:
# mount drive and add path to root folder so custom_GANs python package is available
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.insert(0,'/content/drive/My Drive/generative_models')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [3]:
from pathlib import Path
import os

from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

from custom_GANs.simple_GANs import VanillaGAN, ModelMonitor, CheckpointCleanupCallback


In [4]:
def scale_images(data):
    image = data['image']
    return image / 255


In [ ]:
# Can be skipped


ds = tfds.load('fashion_mnist', split='train')
dataiterator = ds.as_numpy_iterator()
# Setup the subplot formatting
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# Loop four times and get images
for idx in range(4):
    # Grab an image and label
    sample = dataiterator.next()
    # Plot the image using a specific subplot
    ax[idx].imshow(np.squeeze(sample['image']))
    # Appending the image label as the plot title
    ax[idx].title.set_text(sample['label'])

In [5]:
example_images_dir =  Path('/content/drive/My Drive/generative_models').resolve() / 'examples/images'
example_images_dir.mkdir(parents=True, exist_ok=True)
print('Example images save dir: ', example_images_dir)

checkpoint_dir =  Path('/content/drive/My Drive/generative_models').resolve() / 'examples/checkpoints'
checkpoint_dir.mkdir(parents=True, exist_ok=True)
print('Checkpoint save dir: ', checkpoint_dir)

Example images save dir:  /content/drive/My Drive/generative_models/examples/images
Checkpoint save dir:  /content/drive/My Drive/generative_models/examples/checkpoints


In [ ]:
gan = VanillaGAN()

ds = tfds.load('fashion_mnist', split='train')
ds = ds.map(scale_images)
ds = ds.cache()
ds = ds.shuffle(60000)
ds = ds.batch(128)
ds = ds.prefetch(64)

# Define the checkpoint callback with epoch information in the filename

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'vanilla_gan_epoch_{epoch:02d}_d_loss_{d_loss:.2f}.keras'),
    monitor='d_loss',          # Monitor the discriminator loss
    save_best_only=True,      # Save every epoch
    save_weights_only=False,   # Save the full model (including architecture)
    mode='min',                # Minimize the monitored metric
    verbose=1                  # Verbosity level
)


gan.compile(use_default=True)
hist = gan.fit(ds, epochs = 2000, batch_size = 128, callbacks = [ModelMonitor(save_path = example_images_dir), checkpoint_callback, CheckpointCleanupCallback(checkpoint_dir)])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2000
